In [ ]:
#import all neceed libraries\
import pandas as pd
import numpy as np
import os
import re
from bs4 import BeautifulSoup
import nltk
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from string import punctuation
import nltk
import re

In [ ]:
#data preprocessing
df=pd.read_csv(r'/content/comments.csv');
df

,comments
0,plss make ukrainian localisation!!!!!!!!!!!!!!!!!
1,🇺🇦localization 🙏
2,como que 2025😭
3,I love this. This is so so cool. An actual fac...
4,GTA V and GTA VI
...,...
26887,fire
26888,❤❤❤
26889,First!❤
26890,First😂🎉


In [ ]:
#cleaning comments by removing emojis,links and html tags
def clean_comment(comment):
    # Remove emojis
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    comment = emoji_pattern.sub(r'', comment)

    # Remove links
    comment = re.sub(r'http\S+', '', comment)

    # Remove HTML tags
    comment = BeautifulSoup(comment, "html.parser").get_text()
    comment = re.sub(r'<a\s+(?:[^>]*?\s+)?href=(["\'])(.*?)\1[^>]*>(.*?)</a>', r'\3', comment)

    # Remove extra whitespace
    comment = re.sub('\s+', ' ', comment).strip()

    return comment

In [ ]:
df['comments'] = df['comments'].apply(clean_comment)
df['comments']

<ipython-input-40-1391c60438d8>:30: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  comment = BeautifulSoup(comment, "html.parser").get_text()


0        plss make ukrainian localisation!!!!!!!!!!!!!!!!!
1                                             localization
2                                            como que 2025
3        I love this. This is so so cool. An actual fac...
4                                         GTA V and GTA VI
                               ...                        
26887                                                 fire
26888                                                     
26889                                               First!
26890                                                First
26891    Got what <a href=" is all about? Tell us, whic...
Name: comments, Length: 26892, dtype: object

In [ ]:
#Labelling comments
nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()
df['positive'] = df['comments'].apply(lambda x: sia.polarity_scores(x)['pos'])
df['negative'] = df['comments'].apply(lambda x: sia.polarity_scores(x)['neg'])
df['neutral'] = df['comments'].apply(lambda x: sia.polarity_scores(x)['neu'])
df['compound'] = df['comments'].apply(lambda x: sia.polarity_scores(x)['compound'])
score=df['compound'].values
sentiment=[]
for i in score:
    if i>=0.05:
        sentiment.append('Positive')
    elif i<=-0.05:
        sentiment.append('Negative')
    else:
        sentiment.append('Neutral')
df['sentiment']=sentiment

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [ ]:
df

,comments,positive,negative,neutral,compound,sentiment
0,plss make ukrainian localisation!!!!!!!!!!!!!!!!!,0.000,0.0,1.000,0.0000,Neutral
1,localization,0.000,0.0,1.000,0.0000,Neutral
2,como que 2025,0.000,0.0,1.000,0.0000,Neutral
3,I love this. This is so so cool. An actual fac...,0.320,0.0,0.680,0.8195,Positive
4,GTA V and GTA VI,0.000,0.0,1.000,0.0000,Neutral
...,...,...,...,...,...,...
26887,fire,0.000,1.0,0.000,-0.3400,Negative
26888,,0.000,0.0,0.000,0.0000,Neutral
26889,First!,0.000,0.0,1.000,0.0000,Neutral
26890,First,0.000,0.0,1.000,0.0000,Neutral


In [ ]:
df1=df.drop(['positive','negative','neutral','compound'],axis=1)
df1.head()

,comments,sentiment
0,plss make ukrainian localisation!!!!!!!!!!!!!!!!!,Neutral
1,localization,Neutral
2,como que 2025,Neutral
3,I love this. This is so so cool. An actual fac...,Positive
4,GTA V and GTA VI,Neutral


In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
stop_words = stopwords.words('english')
porter_stemmer = PorterStemmer()
lzr = WordNetLemmatizer()

In [ ]:
#removing negative words
stopwords=[
    'no', 'very', "don't", "aren't", "couldn't", "didn't",
    "doesn't", "hadn't", "hasn't", "haven't", "isn't", "mightn't",
    "mustn't", "needn't", "shan't", "shouldn't", "wasn't", "weren't",
    'won', "won't", "wouldn't"
]
stop_words=[word for word in stop_words if word not in stopwords]

In [ ]:
def text_processing(text):
    # convert text into lowercase
    text = text.lower()

    # remove new line characters in text
    text = re.sub(r'\n',' ', text)

    # remove punctuations from text
    text = re.sub('[%s]' % re.escape(punctuation), "", text)

    # remove references and hashtags from text
    text = re.sub("^a-zA-Z0-9$,.", "", text)

    # remove multiple spaces from text
    text = re.sub(r'\s+', ' ', text, flags=re.I)

    # remove special characters from text
    text = re.sub(r'\W', ' ', text)

    text = ' '.join([word for word in word_tokenize(text) if word not in stop_words])

    # stemming using porter stemmer from nltk package - msh a7sn 7aga - momken: lancaster, snowball
    text=' '.join([porter_stemmer.stem(word) for word in word_tokenize(text)])

    # lemmatizer using WordNetLemmatizer from nltk package
    text=' '.join([lzr.lemmatize(word) for word in word_tokenize(text)])

    return text

In [ ]:
#copying the original data
nltk.download('omw-1.4')
nltk.download('punkt')
df_copy=df1.copy()
df_copy['comments']=df_copy['comments'].apply(text_processing)

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
df_copy

,comments,sentiment
0,plss make ukrainian localis,Neutral
1,local,Neutral
2,como que 2025,Neutral
3,love cool actual fact gta vice citi instal tablet,Positive
4,gta v gta vi,Neutral
...,...,...
26887,fire,Negative
26888,,Neutral
26889,first,Neutral
26890,first,Neutral


In [ ]:
df_copy['sentiment'].value_counts()

Neutral     12310
Positive    11092
Negative     3490
Name: sentiment, dtype: int64

In [ ]:
df_copy.to_csv('sentiment.csv')

In [ ]:
df_copy['encoded_sentiment']=df['sentiment'].map({'Positive':2,'Neutral':1,'Negative':0})
df_copy

,comments,sentiment,encoded_sentiment
0,plss make ukrainian localis,Neutral,1
1,local,Neutral,1
2,como que 2025,Neutral,1
3,love cool actual fact gta vice citi instal tablet,Positive,2
4,gta v gta vi,Neutral,1
...,...,...,...
26887,fire,Negative,0
26888,,Neutral,1
26889,first,Neutral,1
26890,first,Neutral,1
